# Load libraries and Themes

In [ ]:
suppressPackageStartupMessages({
    suppressWarnings({
        library(Seurat)
        library(ggplot2)
        library(tidyverse)
        library(SeuratDisk)
        library(DESeq2)
        library(sctransform)
        library(edgeR)
        library(EnhancedVolcano)
        library(Rsamtools)
        library(svglite)
        library(viridis)
        library(pals)
        library(harmony)
        library(WGCNA)
        library(hdWGCNA)
        library(patchwork)
        library(UpSetR)  
        library(ComplexUpset)
        library(sva)
        })})

In [ ]:
#Color Palettes

palette.EC <- c(
    "#8e69a2", # Muted purple  
    "#004c6d", # Teal blue  
    "#006f8e", # Dark teal  
    "#3388b5", # Medium blue  
    "#66a1c3", # Soft blue  
    "#003366" # Deep navy blue  
)

palette.treatment <- c(
    "#C1C1C1", # CTRL
    "#AF0000", # ALDO
    "#006AF3"  # REC
)

palette.sex <- c(
    "#006AF3", 
    "#AF0000" ) #female

In [ ]:
umap_theme <- theme(
  axis.line=element_blank(),
  axis.text.x=element_blank(),
  axis.text.y=element_blank(),
  axis.ticks=element_blank(),
  axis.title.x=element_blank(),
  axis.title.y=element_blank(),
  panel.background=element_blank(),
  panel.border=element_blank(),
  panel.grid.major=element_blank(),
  panel.grid.minor=element_blank()
)

In [ ]:
setwd("/media/daten/dmeral/scseq_analysis/2024_LV_CTRL_ALDO_REC")

In [ ]:
celltype <- "EC"

# Subcluster analysis

In [ ]:
obj <- LoadH5Seurat("seurat_objects/obj_seu_merge_harmony_sgl_addmodule_rename_CMcomb_onlyprotcod_ccscore.h5seurat")
obj_full <- obj

In [ ]:
# Subset to major clusters
obj_sub <- subset(x = obj, subset = cell_type_CMcomb %in% c("EC-cap", "EC-art", "EC-end", "EC-lym"))

In [ ]:
# Get genes with non-zero counts
counts <- GetAssayData(obj_sub, layer = "counts")[,]
nonzero <- as.data.frame(rowSums(counts) > 0)
names(nonzero)[names(nonzero) == "rowSums(counts) > 0"] <- "nonzerofeature"
nonzero <- filter(nonzero, nonzerofeature == TRUE)
nonzero$names <- rownames(nonzero)
nonzero$nonzerofeature <- NULL
write.table(nonzero, paste0("nonzerocounts/nonzerocounts_", celltype, ".csv"), sep = ",", quote = FALSE,  row.names = FALSE, col.names = FALSE)

## EC subcluster

In [ ]:
protein_coding_genes <- unlist(read.csv("DEGs/nothreshold/protein_coding_gene_names_filtered.txt", header = TRUE, stringsAsFactors = FALSE))

# Ensure the protein-coding genes are present in the dataset
selected_genes <- protein_coding_genes[protein_coding_genes %in% rownames(obj_sub)]

# Normalize and scale the data for the selected genes
obj_sub <- ScaleData(obj_sub, features = selected_genes, verbose = FALSE)

# Run PCA using only the selected protein-coding genes
obj_sub <- RunPCA(obj_sub, features = selected_genes, npcs = 35, verbose = FALSE)

In [ ]:
# Integrate "batch"
obj_sub$batch <- as.factor(obj_sub$batch)

obj.subcluster <- obj_sub %>% 
  RunHarmony(group.by.vars = c("batch"), theta = c(2), lambda = c(1), max_iter = 20, early_stop = FALSE, plot_convergence = FALSE, assay.use = "RNA", verbose = FALSE)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 2.5, repr.plot.res = 300) 

dim1 <- DimPlot(obj_sub, reduction = "pca", group.by = "batch", dims = c(1, 2), cols = palette.treatment) + ggtitle("PC batch before harmony") # Before
dim2 <- DimPlot(obj.subcluster, reduction = "harmony", group.by = "batch", dims = c(1, 2), cols = palette.treatment) + ggtitle("PC batch after harmony") # After

ggsave(paste0("subcluster/", celltype, "/Plots/PC_batch_before_harmony.svg"), plot = dim1, units = "cm", dpi = 300, width = 15, height = 10)
ggsave(paste0("subcluster/", celltype, "/Plots/PC_batch_after_harmony.svg"), plot = dim2, units = "cm", dpi = 300, width = 15, height = 10)

In [ ]:
# Run UMAP
obj.subcluster <- obj.subcluster %>%
  RunUMAP(dims = 1:35,  spread = 2, min.dist = 0.6, reduction = "harmony", verbose = FALSE) %>%
  FindNeighbors(dims = 1:35, reduction = "harmony", verbose = FALSE) %>%
  FindClusters(resolution = 0.15)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 300) 

seurat_clusters <- DimPlot(obj.subcluster,  group.by = "seurat_clusters",  pt.size = 1, label = TRUE, shuffle = FALSE, label.size = 10, cols = palette.EC) + umap_theme & NoLegend()
treatment <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", shuffle = TRUE, cols = palette.treatment) + umap_theme & NoLegend()
sex <- DimPlot(obj.subcluster, pt.size = 1,  group.by = "sex", shuffle = TRUE, label.size = 10, , cols = palette.sex) + umap_theme
batch <- DimPlot(obj.subcluster, pt.size = 1,  group.by = "batch", shuffle = TRUE, label.size = 10, cols = palette.treatment) + umap_theme

seurat_clusters|treatment|sex|batch

In [ ]:
# Rename Idents and add cell_type_sub, we rename the subclusters to EC_0-5 
cluster_annotations <- c("Cap_1", "Cap_2", "Cap_vein", "Art", "End", "Lym")
names(cluster_annotations) <- levels(obj.subcluster)
obj.subcluster <- RenameIdents(obj.subcluster, cluster_annotations)
obj.subcluster$cell_type_sub <- Idents(obj.subcluster)

In [ ]:
# Plot and save plots
cell_type_sub <- DimPlot(obj.subcluster, group.by = "cell_type_sub", pt.size = 1, label = TRUE, shuffle = F, label.size = 10, cols = palette.EC) + umap_theme & NoLegend()

ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_cell_type_sub.svg"), plot = cell_type_sub, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_seurat_clusters.svg"), plot = seurat_clusters, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_treatment.svg"), plot = treatment, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_sex.svg"), plot = sex, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_batch.svg"), plot = batch, units = "cm", dpi = 300, width = 18, height = 15)

In [ ]:
treatment <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", shuffle = TRUE, cols = palette.treatment, split.by = "treatment") + umap_theme & NoLegend()
treatment

ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_treatment_split.svg"), plot = treatment, units = "cm", dpi = 300, width = 18*3, height = 15)

In [ ]:
SaveH5Seurat(obj.subcluster, paste0("subcluster/", celltype, "/Subcluster_", celltype))

## FindAllMarkers

In [ ]:
all.markers <- FindAllMarkers(obj.subcluster, only.pos = FALSE, test.use = "wilcox")

write.csv(all.markers, file = paste0("subcluster/", celltype, "/all.markers_" , celltype, "_all.csv"))

## Cellnumbers per cluster and sample

In [ ]:
print("Cellnumbers per cluster and sample")
table <- table(obj.subcluster@meta.data$treatment, 
      obj.subcluster@meta.data$cell_type_sub)
table
write.csv(table, file = paste0("subcluster/", celltype, "/number_perCluster_", celltype, ".csv"))

print("Cellnumbers per sample and sample")
table <- table(obj.subcluster@meta.data$sample_id, 
      obj.subcluster@meta.data$cell_type_sub)
table
write.csv(table, file = paste0("subcluster/", celltype, "/number_persample_", celltype, ".csv"))

# DEG analysis

## WR sum

In [ ]:
# Perform Wilcoxon DEG analysis between CM_0 and CM_1
deg_Cap1_vs_Cap2 <- FindMarkers(
  obj.subcluster, 
  ident.1 = "Cap_1", 
  ident.2 = "Cap_2", 
  group.by = "cell_type_sub",
  test.use = "wilcox"
)

In [ ]:
# Save results to CSV
write.csv(deg_Cap1_vs_Cap2, file = paste0("subcluster/", celltype , "/DEGs_Cap1_vs_Cap2.csv"))

In [ ]:
# Load the list of protein-coding genes
protein_coding_genes <- unlist(read.csv("DEGs/nothreshold/protein_coding_gene_names_filtered.txt", header = TRUE, stringsAsFactors = FALSE))

# Load the DEG file for CM0 vs CM1
deg_Cap1_vs_Cap2 <- read.csv(paste0("subcluster/", celltype, "/DEGs_Cap1_vs_Cap2.csv"), header = TRUE, row.names = 1)

# Add filtering criteria
deg_Cap1_vs_Cap2 <- cbind(deg_Cap1_vs_Cap2, Gene_names = rownames(deg_Cap1_vs_Cap2))

# Filter for protein-coding genes, remove GM... and ...Rik, and non-mt genes
deg_Cap1_vs_Cap2_filtered <- deg_Cap1_vs_Cap2 %>%
  filter(
    Gene_names %in% protein_coding_genes               # Keep only protein-coding genes
  )

# Remove the Gene_names column
deg_Cap1_vs_Cap2_filtered <- deg_Cap1_vs_Cap2_filtered[, !names(deg_Cap1_vs_Cap2_filtered) %in% c("Gene_names")]

# Save the filtered file
write.csv(deg_Cap1_vs_Cap2_filtered, file = paste0("subcluster/", celltype, "/DEGs_Cap1_vs_Cap2_filtered.csv"))

## pseudo-bulk PCA plot

In [ ]:
# Aggregate counts by `sample_id`
obj.subcluster$tsid <- paste0(obj.subcluster$treatment, "_", obj.subcluster$sample_id)

cts <- AggregateExpression(
  obj.subcluster,
  group.by = "tsid",  # Only group by combined column tsid
  assays = "RNA",
  slot = "counts",
  return.seurat = FALSE
)

cts <- cts$RNA  # Extract the counts matrix

# Filter counts for only protein-coding genes
cts_filtered <- cts[rownames(cts) %in% protein_coding_genes, ]

# Log-transform counts (avoid log(0) issues)
log_counts <- log1p(cts_filtered)

# Remove zero-variance genes
log_counts_filtered <- log_counts[apply(log_counts, 1, var) > 0, ]

# Ensure that `tsid` is correctly defined in the metadata
# Extract the `batch_info` corresponding to the 12 aggregated samples
batch_info <- obj.subcluster@meta.data$batch[match(colnames(log_counts_filtered), obj.subcluster@meta.data$tsid)]

# Add the batch info to the log_counts_filtered matrix as an additional row
log_counts_filtered_with_batch <- rbind(log_counts_filtered, batch = batch_info)

# Now, apply ComBat for batch effect correction using the log-transformed counts with batch information
adjusted_counts <- ComBat(
    dat = log_counts_filtered_with_batch[1:nrow(log_counts_filtered), ],  # Only include gene expression matrix
    batch = log_counts_filtered_with_batch["batch", ],  # Use the batch info row
    mod = NULL,                   # Model matrix 
    par.prior = TRUE,             # Use empirical Bayes method
    prior.plots = FALSE           # Don't plot the correction
)

# Perform PCA on the adjusted counts
pca_result <- prcomp(t(adjusted_counts), center = TRUE, scale. = TRUE)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)

# Create a data frame for plotting
pca_df <- data.frame(
  PC1 = pca_result$x[, 1],  # First principal component
  PC2 = pca_result$x[, 2],  # Second principal component
  tsid = rownames(pca_result$x),  # Use rownames of pca_result$x for tsid
  treatment = gsub("_.*", "", rownames(pca_result$x))  # Extract treatment from tsid
)

# Ensure treatment is a factor with levels in the correct order
pca_df$treatment <- factor(pca_df$treatment, levels = c("CTRL", "ALDO", "REC"))

# Plot the PCA
PCA_plot <- ggplot(pca_df, aes(x = PC1, y = PC2, color = treatment, label = substr(tsid, nchar(tsid)-3, nchar(tsid)))) +
  geom_point(size = 4, alpha = 0.8) +
  geom_text(vjust = 2, hjust = 0.5, size = 3, color = "black") +  # Adjust vjust to place labels below
  theme_minimal() +
  labs(
    title = paste0("PCA of Pseudobulk Samples for ", celltype ," Post-ComBat Correction"),
    x = paste0("PC1 (", round(summary(pca_result)$importance[2, 1] * 100, 1), "% Variance)"),
    y = paste0("PC2 (", round(summary(pca_result)$importance[2, 2] * 100, 1), "% Variance)")
  ) +
  scale_color_manual(values = palette.treatment) +  # Correct color palette
  theme(
    legend.position = "right",
    text = element_text(size = 12),
    plot.title = element_text(size = 12),  # Decrease title size
    panel.grid.minor = element_blank(),    # Remove minor gridlines only
    panel.border = element_rect(color = "black", fill = NA, size = 0.5)  # Add black border around the plot
  )

PCA_plot
ggsave(paste0("subcluster/", celltype, "/Plots/PCA_plot_pseudobulk.svg"), plot = PCA_plot, units = "cm", dpi = 300, width = 15, height = 10)

# hdWGCNA

In [ ]:
print("Cellnumbers per cluster and sample")
table <- table(obj.subcluster@meta.data$sample_id, 
      obj.subcluster@meta.data$cell_type_sub)
table

In [ ]:
# List with only protein-coding Genes
protein_coding_genes <- read.csv("DEGs/nothreshold/protein_coding_gene_names_filtered.txt", 
                                 header = FALSE, 
                                 stringsAsFactors = FALSE)[,1]

protein_coding_genes <- protein_coding_genes[-1]

In [ ]:
# Filter the gene expression matrix in the Seurat object
seurat_obj <- obj.subcluster
seurat_obj <- subset(seurat_obj, features = protein_coding_genes)  

# Now proceed with WGCNA setup
seurat_obj <- SetupForWGCNA(
  seurat_obj,
  gene_select = "fraction",  # You can still use fraction-based selection
  fraction = 0.05,          
  wgcna_name = celltype
)

In [ ]:
# Construct metacells in each group (make sure that min_cells>k)
seurat_obj <- MetacellsByGroups(
  seurat_obj = seurat_obj,
  group.by = c("treatment", "cell_type_sub"),
  reduction = "harmony", 
  assay = "RNA",
  slot = "counts",
  k = 12,                          
  max_shared = 20,         
  ident.group = "cell_type_sub",      
  min_cells = 13
)

In [ ]:
# Normalize metacell expression matrix:
seurat_obj <- NormalizeMetacells(seurat_obj)

seurat_obj <- SetDatExpr(
  seurat_obj,
  group_name = unique(obj.subcluster@meta.data$cell_type_sub),  # cluster "Cap_1", "Cap_2", "Cap_vein", "Art", "End", "Lym"
  group.by = "cell_type_sub",                                  # the metadata column containing the cell type info. This same column should have also been used in MetacellsByGroups
  assay = "RNA",                                               # using RNA assay
  slot = "data"                                                # using normalized data
)

# Test different soft powers:
seurat_obj <- TestSoftPowers(
  seurat_obj,
  networkType = "signed")

# Plot the results:
plot_list <- PlotSoftPowers(seurat_obj)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)

# assemble with patchwork
wrap_plots(plot_list, ncol = 2)

In [ ]:
# construct co-expression network:
seurat_obj <- ConstructNetwork(
  seurat_obj, soft_power = 6,
  setDatExpr = FALSE,
  tom_name = paste0("LV_", celltype), overwrite_tom = TRUE)

In [ ]:
ls()

In [ ]:
# Generate palette
palette_mod <- DiscretePalette(6, palette = "stepped", shuffle = TRUE)

# Function to check if a color is grey (low saturation)
is_grey <- function(color) {
  hsv_vals <- rgb2hsv(col2rgb(color))
  return(hsv_vals["s", ] < 0.2)  # Low saturation indicates grey
}

# Filter out greys
palette_mod <- palette_mod[!sapply(palette_mod, is_grey)]

# If too many colors were removed, regenerate until enough remain
while (length(palette_mod) < 6) {
  new_colors <- DiscretePalette(6, palette = "stepped", shuffle = TRUE)
  palette_mod <- unique(c(palette_mod, new_colors[!sapply(new_colors, is_grey)]))
  palette_mod <- palette_mod[1:6]  # Ensure exactly 9 colors
}

# Print final colors
print(palette_mod)

In [ ]:
# Reset the module colors
seurat_obj <- ResetModuleColors(seurat_obj, palette_mod)

In [ ]:
svg(paste0("subcluster/", celltype, "/hdWGCNA/Dendrogram.svg"), width = 15 / 2.54, height = 10 / 2.54)  # Convert cm to inches
PlotDendrogram(seurat_obj, main = paste0(celltype, " hdWGCNA Dendrogram"))
dev.off()

In [ ]:
TOM <- GetTOM(seurat_obj)
head(TOM)

In [ ]:
# need to run ScaleData first or else harmony throws an error:
seurat_obj <- ScaleData(seurat_obj, features = VariableFeatures(seurat_obj))

# compute all MEs in the full single-cell dataset
seurat_obj <- ModuleEigengenes(
 seurat_obj,
 group.by.vars = "cell_type_sub")

In [ ]:
# harmonized module eigengenes:
hMEs <- GetMEs(seurat_obj)

# module eigengenes:
MEs <- GetMEs(seurat_obj, harmonized = FALSE)

In [ ]:
# compute eigengene-based connectivity (kME):
seurat_obj <- ModuleConnectivity(
  seurat_obj,
  group.by = "cell_type_sub", group_name = c("Cap_1", "Cap_2", "Cap_vein", "Art", "End", "Lym"))

In [ ]:
# rename the modules
seurat_obj <- ResetModuleNames(
  seurat_obj,
  new_name = paste0(celltype ,"-M")
)

In [ ]:
saveRDS(seurat_obj, file = paste0("subcluster/", celltype, "/hdWGCNA/hdWGCNA_" , celltype, ".rds"))

In [ ]:
svg(paste0("subcluster/", celltype, "/hdWGCNA/KMEs.svg"), width = 15 / 2.54, height = 15 / 2.54)  # Convert cm to inches
PlotKMEs(seurat_obj, ncol = 3)
dev.off()

In [ ]:
# get the module assignment table:
modules <- GetModules(seurat_obj)

# show the first 6 columns:
head(modules[,1:8])
write.csv(modules, file = paste0("subcluster/", celltype, "/hdWGCNA/modules_df.csv"))

In [ ]:
# get hub genes
hub_df <- GetHubGenes(seurat_obj, n_hubs = 25)

head(hub_df)
write.csv(hub_df, file = paste0("subcluster/", celltype, "/hdWGCNA/hub_df.csv"))

In [ ]:
# make a featureplot of hMEs for each module
plot_list <- ModuleFeaturePlot(
  seurat_obj,
  features = "hMEs", # plot the hMEs
  order = TRUE # order so the points with highest hMEs are on top
)
# stitch together with patchwork
combined_plot <- wrap_plots(plot_list, ncol = 3)

ggsave(
  filename = paste0("subcluster/", celltype, "/hdWGCNA/ModuleFeaturePlot.svg"),
  plot = combined_plot,  # The patchwork object
  width = 15 / 2.54, 
  height = 15 / 2.54, 
  units = "in",  # Inches (ggsave doesn't use cm natively)
  dpi = 300
)

In [ ]:
# Run module UMAP
seurat_obj <- RunModuleUMAP(
  seurat_obj,
  n_hubs = 10, # number of hub genes to include for the UMAP embedding
  n_neighbors = 15, # neighbors parameter for UMAP
  min_dist = 0.1 # min distance between points in UMAP space
)

# get the hub gene UMAP table from the seurat object
umap_df <- GetModuleUMAP(seurat_obj)

In [ ]:
# All Genes
svg(paste0("subcluster/", celltype, "/hdWGCNA/ModuleUMAPPlot.svg"), width = 15 / 2.54, height = 15 / 2.54)  # Convert cm to inches

UMAPmodule <- ModuleUMAPPlot(
  seurat_obj,
  edge.alpha = 0.25,
  sample_edges = TRUE,
  edge_prop = 0.05, # proportion of edges to sample (20% here)
  label_hubs = 2 ,# how many hub genes to plot per module?
  keep_grey_edges = FALSE
)

dev.off()

#### DMEs

In [ ]:
group1 <- seurat_obj@meta.data %>% subset(treatment == "CTRL") %>% rownames
group2 <- seurat_obj@meta.data %>% subset(treatment == "ALDO") %>% rownames
group3 <- seurat_obj@meta.data %>% subset(treatment == "REC") %>% rownames

##### ALDO vs CTRL

In [ ]:
DMEs <- FindDMEs(
  seurat_obj,
  barcodes1 = group2,
  barcodes2 = group1,
  test.use = "wilcox",
  wgcna_name = celltype
)

DMEs

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 100)

p <- PlotDMEsLollipop(
  seurat_obj, 
  DMEs, 
  wgcna_name = celltype, 
  pvalue = "p_val_adj"
)

ggsave(paste0("subcluster/", celltype, "/hdWGCNA/PlotDMEsLollipop_ALDOvsCTRL.svg"), plot = p, units = "cm", dpi = 300, width = 10, height = 10)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 100)

p <- PlotDMEsVolcano(
  seurat_obj,
  DMEs,
  wgcna_name = celltype
)

p

##### REC vs ALDO

In [ ]:
DMEs <- FindDMEs(
  seurat_obj,
  barcodes1 = group3,
  barcodes2 = group2,
  test.use = "wilcox",
  wgcna_name = celltype
)

DMEs

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 100)

p <- PlotDMEsLollipop(
  seurat_obj, 
  DMEs, 
  wgcna_name = celltype, 
  pvalue = "p_val_adj"
)

ggsave(paste0("subcluster/", celltype, "/hdWGCNA/PlotDMEsLollipop_RECvsALDO.svg"), plot = p, units = "cm", dpi = 300, width = 10, height = 10)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 100)

p <- PlotDMEsVolcano(
  seurat_obj,
  DMEs,
  wgcna_name = celltype
)

p

In [52]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.2 LTS

Matrix products: default
BLAS/LAPACK: /media/daten/dmeral/micromamba/envs/user_R/lib/libopenblasp-r0.3.26.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] ggforestplot_0.1.0          sva_3.46.0                 
 [3] BiocParallel_1.32.5         genefilter_1.80.3          
 [5] mgcv_1.9-1                  nlme_3.1-164               
 [7] ComplexUpset_1.3.3          UpSetR_1.4.0               
 [9] patchwork_1.3.0             hdWGCN